# Script #6 - Data Mapper (GEOS Projection)

*Mike Huff, 2025*

https://github.com/m-huff

If you intend to keep your data in the GEOS projection and want to reproject other datasets to this projection (like in the "What GOES Around" map on my website, huffmaps.com), this is where the magic happens. This script does require you to download the "natural earth" country boundaries from this link:

https://www.naturalearthdata.com/downloads/110m-cultural-vectors/

This script prepares and visualizes your GEOTIFF, using an example GOES NetCDF file (note: use one of the original ones you downloaded straight from the NASA EarthData Downloader - this script looks inside and steals the projection data from there), country boundaries, and a custom graticule (latitude-longitude grid) to create geospatial reference layers that align with the GOES satellite view. 

The `grid_spacing` parameter in the code cell below can be changed to customize the interval at which lat/long graticules are shown.

The script first extracts the GOES projection parameters (e.g., satellite height, sweep angle, and projection origin) from the NetCDF file to build a valid geostationary CRS definition. Then, it loads a global country shapefile, defines the satellite’s visible disk (the Earth area observable from the GOES position), and clips the country boundaries to that circular extent. A graticule grid is generated only within this visible region, ensuring that gridlines follow the satellite’s true field of view. Both layers (the clipped countries and graticule) are reprojected into the GOES CRS and saved as shapefiles for later use in map overlays. Finally, the script displays a quick verification plot that overlays the GOES-referenced raster data, clipped countries, and graticule to confirm alignment.

You can also use the `SHOW_GRATICULES_IN_OUTPUT`, `SHOW_COUNTRIES_IN_OUTPUT`, and `SHOW_GOES_DATA_IN_OUTPUT` variables to change which layers are shown in the output plot (appears below the code cell after running the script). In making the "What GOES Around" map, I actually just exported each layer (the graticules, the countries, and the GOES data) separately from the generated plot and created the map layout in Adobe InDesign. No shapefiles or even any GIS needed!

How to use:

1. Update the file paths for your untouched example GOES NetCDF (again, one that came straight from the NASA EarthData Downloader, not the data GEOTIFF you've been building over the course of these scripts), the natural earth country shapefile, and output directory.
2. Adjust grid_spacing (in degrees) to control the density of graticule lines.
3. The script will produce GOES-projected shapefiles for countries and graticules, saved in your output directory, and display a verification map for visual confirmation.

In [ ]:
%pip install rasterio matplotlib xarray geopandas shapely

import xarray as xr
import geopandas as gpd
from shapely.geometry import LineString, Point, Polygon
import numpy as np
import os
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

### VARIABLES TO CONTROL THE SCRIPT
### THESE ARE DESCRIBED IN THE MARKDOWN CELL ABOVE
netcdf_path = r"E:\GOES-R Lightning Data\WEST-2025\OR_GLM-L3-GLMF-M6_G18_s202518123590000_e202518200000000_c20251820001000.nc"
raster_path = r"E:\GOES-R Lightning Data\WEST-RASTERS\west_mean_energy_2025_georef.tif"
countries_path = r"E:\GOES-R Lightning Data\GEOS-REPROJECT-LAYERS\countries-natural-earth\ne_110m_admin_0_countries.shp"
output_dir = r"E:\GOES-R Lightning Data\GEOS-REPROJECT-LAYERS"
grid_spacing = 10  
SHOW_GRATICULES_IN_OUTPUT = True
SHOW_COUNTRIES_IN_OUTPUT = True
SHOW_GOES_DATA_IN_OUTPUT = True

ds = xr.open_dataset(netcdf_path)
proj_var = ds['goes_imager_projection']
h = float(proj_var.perspective_point_height)
lon_0 = float(proj_var.longitude_of_projection_origin)
sweep = str(proj_var.sweep_angle_axis)
a = float(proj_var.semi_major_axis)
b = float(proj_var.semi_minor_axis)

goes_crs = f"+proj=geos +h={h} +lon_0={lon_0} +sweep={sweep} +a={a} +b={b} +units=m +no_defs"
print(f"GOES CRS:\n{goes_crs}")

world = gpd.read_file(countries_path)

lat_max = 81
lon_max = 81
num_points = 360
angles = np.linspace(0, 2*np.pi, num_points)
disk_lon = lon_0 + lon_max * np.cos(angles)
disk_lat = 0 + lat_max * np.sin(angles)
visible_disk = Polygon(zip(disk_lon, disk_lat))
visible_gdf = gpd.GeoDataFrame(geometry=[visible_disk], crs="EPSG:4326")

world_clipped = gpd.clip(world, visible_gdf)

lats = np.arange(-lat_max, lat_max + 1, grid_spacing)
lons = np.arange(lon_0 - lon_max, lon_0 + lon_max + 1, grid_spacing)

lines = []
for lon in lons:
    pts = [Point(lon, lat) for lat in lats if visible_disk.contains(Point(lon, lat))]
    if len(pts) >= 2:
        lines.append(LineString(pts))

for lat in lats:
    pts = [Point(lon, lat) for lon in lons if visible_disk.contains(Point(lon, lat))]
    if len(pts) >= 2:
        lines.append(LineString(pts))

graticule = gpd.GeoDataFrame(geometry=lines, crs="EPSG:4326")

world_goes = world_clipped.to_crs(goes_crs)
graticule_goes = graticule.to_crs(goes_crs)

os.makedirs(output_dir, exist_ok=True)
world_out = os.path.join(output_dir, "world_borders_geos.shp")
grid_out = os.path.join(output_dir, f"graticule_{grid_spacing}deg_geos.shp")

world_goes.to_file(world_out)
graticule_goes.to_file(grid_out)

print(f"\nShapefiles saved:\n{world_out}\n{grid_out}")

fig, ax = plt.subplots(figsize=(15, 15))

if (SHOW_GRATICULES_IN_OUTPUT):
    graticule_goes.plot(ax=ax, color='lightgray', linewidth=0.5)

if (SHOW_COUNTRIES_IN_OUTPUT):    
    world_goes.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1)

ax.set_title(f"Countries and Graticule in GOES Projection ({lon_0}° subsatellite)")

with rasterio.open(raster_path) as src:
    raster_data = src.read(1)
    raster_extent = (
        src.bounds.left, src.bounds.right,
        src.bounds.bottom, src.bounds.top
    )

if (SHOW_GOES_DATA_IN_OUTPUT):
    show(raster_data, extent=raster_extent, ax=ax, cmap='pink', alpha=0.9)

plt.axis('equal')
plt.show()